- Quantidade de viagens iniciadas por estação
- Quantidade de viagens finalizadas por estação
- Qual a média de duração de uma viagem com bike do tipo eletric e classic?


In [20]:
file_path="/home/jovyan/work/data/202212_divvy_tripdata.csv"

#Cria sessão para ambiente local
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("manipulando-dados") \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

df = spark.read.option("header", "true").csv(file_path)
df.limit(5)

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
65DBD2F447EC51C2,electric_bike,2022-12-05 10:47:18,2022-12-05 10:56:34,Clifton Ave & Arm...,TA1307000163,Sedgwick St & Web...,13191,41.9182435,-87.65711466666667,41.922167,-87.638888,member
0C201AA7EA0EA1AD,classic_bike,2022-12-18 06:42:33,2022-12-18 07:08:44,Broadway & Belmon...,13277,Sedgwick St & Web...,13191,41.940106,-87.645451,41.922167,-87.638888,casual
E0B148CCB358A49D,electric_bike,2022-12-13 08:47:45,2022-12-13 08:59:51,Sangamon St & Lak...,TA1306000015,St. Clair St & Er...,13016,41.885918856,-87.65113318,41.89434513742426,-87.62279838323593,member
54C5775D2B7C9188,classic_bike,2022-12-13 18:50:47,2022-12-13 19:19:48,Shields Ave & 31s...,KA1503000038,Damen Ave & Madis...,13134,41.838464,-87.635406,41.88137,-87.67493,member
A4891F78776D35DF,classic_bike,2022-12-14 16:13:39,2022-12-14 16:27:50,Ashland Ave & Chi...,13247,Damen Ave & Charl...,13288,41.89595435734305,-87.6677280664444,41.920082,-87.677855,casual


In [22]:
from pyspark.sql.functions import upper, expr, split, col

result_df_py = (
  df
  .withColumn("start_station_name", upper(df["start_station_name"]))
  .withColumn("end_station_name", upper(df["end_station_name"]))
  .withColumn("trip_time", (expr("unix_timestamp(ended_at) - unix_timestamp(started_at)") / 60).cast("int"))
  .withColumn("rideable_type", split("rideable_type", "_")[0])
  .withColumnRenamed("member_casual", "member_category")
  
)

result_df_py.limit(5)


ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_category,trip_time
65DBD2F447EC51C2,electric,2022-12-05 10:47:18,2022-12-05 10:56:34,CLIFTON AVE & ARM...,TA1307000163,SEDGWICK ST & WEB...,13191,41.9182435,-87.65711466666667,41.922167,-87.638888,member,9
0C201AA7EA0EA1AD,classic,2022-12-18 06:42:33,2022-12-18 07:08:44,BROADWAY & BELMON...,13277,SEDGWICK ST & WEB...,13191,41.940106,-87.645451,41.922167,-87.638888,casual,26
E0B148CCB358A49D,electric,2022-12-13 08:47:45,2022-12-13 08:59:51,SANGAMON ST & LAK...,TA1306000015,ST. CLAIR ST & ER...,13016,41.885918856,-87.65113318,41.89434513742426,-87.62279838323593,member,12
54C5775D2B7C9188,classic,2022-12-13 18:50:47,2022-12-13 19:19:48,SHIELDS AVE & 31S...,KA1503000038,DAMEN AVE & MADIS...,13134,41.838464,-87.635406,41.88137,-87.67493,member,29
A4891F78776D35DF,classic,2022-12-14 16:13:39,2022-12-14 16:27:50,ASHLAND AVE & CHI...,13247,DAMEN AVE & CHARL...,13288,41.89595435734305,-87.6677280664444,41.920082,-87.677855,casual,14


- Quantidade de viagens iniciadas por estação

In [28]:
df_station_agg = result_df_py.groupBy("start_station_name").count().orderBy("count", ascending=False)
display(df_station_agg)

start_station_name,count
NULL,29283
KINGSBURY ST & KI...,1422
CLINTON ST & WASH...,1330
CLARK ST & ELM ST,1226
STATE ST & CHICAG...,1180
CLINTON ST & MADI...,1067
CANAL ST & ADAMS ST,1051
WELLS ST & CONCOR...,1041
WELLS ST & ELM ST,1028
WELLS ST & HURON ST,1025


In [32]:
from pyspark.sql.functions import when, col

In [39]:
df_fill_start_station = result_df_py\
    .withColumn("start_station_name", when(col("start_station_name").isNull(),"unknown").otherwise(result_df_py["start_station_name"]) )


df_station_agg = df_fill_start_station.groupBy("start_station_name").count().orderBy("count", ascending=False)

display(df_station_agg)

start_station_name,count
unknown,29283
KINGSBURY ST & KI...,1422
CLINTON ST & WASH...,1330
CLARK ST & ELM ST,1226
STATE ST & CHICAG...,1180
CLINTON ST & MADI...,1067
CANAL ST & ADAMS ST,1051
WELLS ST & CONCOR...,1041
WELLS ST & ELM ST,1028
WELLS ST & HURON ST,1025


- Quantidade de viagens finalizadas por estação


In [43]:
df_fill_end_station = result_df_py\
    .withColumn("end_station_name", when(col("end_station_name").isNull(),"unknown").otherwise(result_df_py["end_station_name"]) )


df_station_agg_end = df_fill_end_station.groupBy("end_station_name").count().orderBy("count", ascending=False)

display(df_station_agg_end)

end_station_name,count
unknown,31158
KINGSBURY ST & KI...,1422
CLINTON ST & WASH...,1402
STATE ST & CHICAG...,1179
CLARK ST & ELM ST,1174
CLINTON ST & MADI...,1144
CANAL ST & ADAMS ST,1115
WELLS ST & CONCOR...,1081
WABASH AVE & GRAN...,1041
WELLS ST & ELM ST,1025


- Qual a média de duração de uma viagem com bike do tipo eletric e classic?

In [44]:
df_avg_trip = result_df_py.groupBy("rideable_type").avg("trip_time")
display(df_avg_trip)

rideable_type,avg(trip_time)
docked,205.70909090909092
classic,13.134383094751193
electric,9.447437834996386


# Desafio
- Reescrever utilizando SQL